# DateTimes

### Introduction

Deciding which features should be included and focused on in our linear model is an important skill of any data scientist.  As we saw previously, if we include features which are too collinear, we will improperly measure the coefficients related to our collinear features.  In addition, feature selection and prioritizing features with feature importance will help us to understand which features to devote our attention to in terms of feature engineering and domain understanding.  Finally, limiting the number of features in our model, and identifying the most crucial features in our model will make our models, and their insights more understandable.

### Working with AirBnb

For this lesson, we'll work with [AirBnb listings in Berlin](https://www.kaggle.com/brittabettendorf/berlin-airbnb-data).

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/mhan1/Data-Science/master/listings_summary.csv')

In [2]:
df.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,2015,https://www.airbnb.com/rooms/2015,20181107122246,2018-11-07,Berlin-Mitte Value! Quiet courtyard/very central,Great location! 30 of 75 sq meters. This wood...,A+++ location! This „Einliegerwohnung“ is an e...,Great location! 30 of 75 sq meters. This wood...,none,It is located in the former East Berlin area o...,...,t,NaN,NaN,f,f,strict_14_with_grace_period,f,f,4,3.76
1,2695,https://www.airbnb.com/rooms/2695,20181107122246,2018-11-07,Prenzlauer Berg close to Mauerpark,NaN,In the summertime we are spending most of our ...,In the summertime we are spending most of our ...,none,NaN,...,t,NaN,NaN,f,f,flexible,f,f,1,1.42


In [3]:
df.shape

(22552, 96)

As we can see, our dataset as 22,500 rows and close to 100 features.  Our goal is to predict the price.

In [4]:
df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'access', 'interaction', 'house_rules',
       'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url',
       'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms',

Some of these columns include data that we do not know how to handle - mainly textual data and images.  A lot of this data is simply not in the correct format.  Let's take a look.

In [5]:
pd.set_option('display.max_rows',100)

A lot of the data is listed as objects, which is really a catch all.

In [6]:
df.dtypes[0:15]

id                        int64
listing_url              object
scrape_id                 int64
last_scraped             object
name                     object
summary                  object
space                    object
description              object
experiences_offered      object
neighborhood_overview    object
notes                    object
transit                  object
access                   object
interaction              object
house_rules              object
dtype: object

In [7]:
def find_object_features(df):
    return list(df.dtypes[df.dtypes == 'object'].index)

In [8]:
len(find_object_features(df))

62

In [9]:
df.dtypes[df.dtypes == 'object'].index

Index(['listing_url', 'last_scraped', 'name', 'summary', 'space',
       'description', 'experiences_offered', 'neighborhood_overview', 'notes',
       'transit', 'access', 'interaction', 'house_rules', 'picture_url',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'is_location_exact',
       'property_type', 'room_type', 'bed_type', 'amenities', 'price',
       'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee',
       'extra_people', 'calendar_updated', 'has_availability',
       'calendar_last_scraped', 'first_review', 'last_review',


In [10]:
find_object_features(df)

['listing_url',
 'last_scraped',
 'name',
 'summary',
 'space',
 'description',
 'experiences_offered',
 'neighborhood_overview',
 'notes',
 'transit',
 'access',
 'interaction',
 'house_rules',
 'picture_url',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'is_location_exact',
 'property_type',
 'room_type',
 'bed_type',
 'amenities',
 'price',
 'weekly_price',
 'monthly_price',
 'security_deposit',
 'cleaning_fee',
 'extra_people',
 'calendar_updated',
 'has_availability',
 'calendar_last_scraped',
 'first_review',
 'last_review',
 'requires_license',
 'license',
 'instant_

### Feature engineering

Let's try to capture as much of this object data as possible.  In this lesson, we'll start with date values.

In [11]:
def find_object_feature_values(df):
    object_features = find_object_features(df)
    return df[object_features][:2].values

In [12]:
find_object_feature_values(df)[0, 1:2]

array(['2018-11-07'], dtype=object)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22552 entries, 0 to 22551
Data columns (total 96 columns):
id                                  22552 non-null int64
listing_url                         22552 non-null object
scrape_id                           22552 non-null int64
last_scraped                        22552 non-null object
name                                22493 non-null object
summary                             21589 non-null object
space                               14020 non-null object
description                         22349 non-null object
experiences_offered                 22552 non-null object
neighborhood_overview               11540 non-null object
notes                               7215 non-null object
transit                             13036 non-null object
access                              10837 non-null object
interaction                         10406 non-null object
house_rules                         11449 non-null object
thumbnail_url           

In [14]:
find_object_feature_values(df)[0, :3]

array(['https://www.airbnb.com/rooms/2015', '2018-11-07',
       'Berlin-Mitte Value! Quiet courtyard/very central'], dtype=object)

We can see that the second column, `last_scraped` should be a date, but is listed as an object.  It would be nice if we could use the `to_datetime` method to put our columns in the correct format.

In [15]:
pd.to_datetime(df, infer_datetime_format=True)

ValueError: to assemble mappings requires at least that [year, month, day] be specified: [day,month,year] is missing

But it doesn't work as well as we would hope. Let's write a method called `is_date` that detects if our column value is a date.

In [16]:
def contains_date(column):
    regex_string = (r'^\d{1,2}-\d{1,2}-\d{4}$|^\d{4}-\d{1,2}-\d{1,2}$' + 
'|^\d{1,2}\/\d{1,2}\/\d{4}$|^\d{4}\/\d{1,2}\/\d{1,2}$')
    return column.str.contains(regex_string).any()

The regex above is wild but it's easier to understand once we understand that each `|` means `or`.  So really, we can think of the above regex as multiple different ones.  So the first one of them is:

`^\d{1,2}-\d{1,2}-\d{4}$`

Which means start with one or two digits, then one or two digits again, and then four digits.  With each set of digits separated by a hyphen.

`11-22-2018`

So contains date tells us if any column in the above contains a date.

In [17]:
def find_date_features(df):
    series_contains_date = df.apply(contains_date)
    return series_contains_date.index[series_contains_date.values]

In [18]:
date_features = find_date_features(df)

In [19]:
date_features

Index(['last_scraped', 'host_since', 'calendar_last_scraped', 'first_review',
       'last_review'],
      dtype='object')

Now we can set those features to be date time.

In [20]:
def to_dates(df):
    date_features = find_date_features(df)
    return df[date_features].astype('datetime64[ns]')

In [21]:
df_date_features = to_dates(df)

In [22]:
df_date_features.dtypes

# last_scraped             datetime64[ns]
# host_since               datetime64[ns]
# calendar_last_scraped    datetime64[ns]
# first_review             datetime64[ns]
# last_review              datetime64[ns]

df_date_features[:1]

,last_scraped,host_since,calendar_last_scraped,first_review,last_review
0,2018-11-07,2008-08-18,2018-11-07,2016-04-11,2018-10-28


In [23]:
from date_lib import add_datepart

In [24]:
def generate_new_date_columns(dates_df):
    copied_dates_df = dates_df.copy()
    for col in copied_dates_df.columns:
        add_datepart(copied_dates_df, col)
    return copied_dates_df

In [25]:
new_date_col_df = generate_new_date_columns(df_date_features)

In [26]:
len(new_date_col_df.columns)

65

Once coercing our dates, we can merge with the original dataframe, and replace each of the old columns that should be dates.

In [27]:
def merge_dfs(original_df, new_df):
    copied_original = original_df.copy()
    date_features = find_date_features(original_df)
    copied_dropped = copied_original.drop(columns = date_features)
    copied_dropped[new_df.columns] = new_df
    return copied_dropped

In [28]:
new_df = merge_dfs(df, new_date_col_df)

We can confirm that the `new_df` has fewer `object_feature` columns than the original.

In [29]:
# new_df = new_df.drop(columns = remaining_date_features)
len(find_object_features(new_df))

57

In [30]:
len(find_object_features(df))

62

Even though our `new_df` has more features.

In [31]:
len(new_df.columns)

156

### Working with Trickier Dates

Sometimes we'll have have some features that represents a date, but is worded as text.  For example, the text '6 years ago' could be translated into a date.  Let's write a function that finds this features with this text.  And also write functions to coerce this data into the proper format.

In [32]:
def contains_time_ago(column):
    regex_string = (r'^\d.*ago$')
    return column.str.contains(regex_string).any()

In [33]:
def find_time_ago_features(df):
    series_contains_time_ago = df.apply(contains_time_ago)
    return series_contains_time_ago.index[series_contains_time_ago.values]

In [34]:
find_time_ago_features(df)

Index(['calendar_updated'], dtype='object')

In [35]:
from date_lib import get_past_date

In [36]:
calendar_updated = df.calendar_updated.apply(get_past_date).astype('datetime64[ns]')

In [37]:
calendar_updated[0:3]

0   2019-04-16
1   2019-05-28
2   2019-07-09
Name: calendar_updated, dtype: datetime64[ns]

Now we can replace our `calendar_updated` column.

### Generating New Features

There are some additional features that we did not generate.  For example, subtracting date columns from another and also subtracting dates from additional key dates like holidays.  Finally, to be even more ambitious, we could match dates to past temperatures if we believe this would help our dataset.

### Summary